In [1]:
import os

In [2]:
%pwd

'/Users/abhishekkumar/Desktop/NLP/UdemyNLP/MLOps/MLFlow/datascienceproject/research'

In [3]:
os.chdir("../")
%pwd

'/Users/abhishekkumar/Desktop/NLP/UdemyNLP/MLOps/MLFlow/datascienceproject'

In [4]:
from dataclasses import dataclass
from pathlib import Path
from src.datascience.constants import * 
from src.datascience.utils.common import read_yaml, create_directories

@dataclass
class DataIngestionconfig:
    root_dir:Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

class ConfiurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH,
                 param_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(param_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionconfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionconfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir  
        )
        return data_ingestion_config

In [5]:
## Component - Data Ingestion
import urllib.request as request
from src.datascience import logger
import zipfile
class DataIngestion:
    def __init__(self, config: DataIngestionconfig):
        self.config = config
    ## Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"file already exists")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [6]:
try:
    config = ConfiurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-12 20:16:48,788: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-12 20:16:48,818: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-12 20:16:48,833: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-12 20:16:48,834: INFO: common: Created directory at: artifacts]
[2025-03-12 20:16:48,836: INFO: common: Created directory at: artifacts/data_ingestion]
[2025-03-12 20:16:49,372: INFO: 1906419236: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F15F:F03F6:18B6AD:3A5F62:67D19DD3
Accept-Ranges: bytes
Date: Wed, 12 